In [1]:
#Standard Imports
import os
import sys
import pandas as pd
import numpy as np
# from collections import defaultdict
import timeit
import argparse
import logging
import copy
import numpy.random as rnd

sys.path.append('../')

# Custom Imports
from src.chord import Chord
from src.musical_work_input import MusicalWorkInput, Harmony
from src.midi_processing import *
# from src.learning_weights import *

import src.evaluate
import src.music_functions
import src.ga_model

In [2]:
# Importing Musical Corpus
#musical_work_df = pd.read_csv("../data/test_melody_hatikvah(israel)_4_1_minor_1.mid")
#musical_corpus = []
#for i, title, meter, key, tonality, first_on_beat, melody in musical_work_df.itertuples():
#    musical_corpus.append(MusicalWorkInput(title, meter, key, tonality, first_on_beat, [int(x) for x in melody.split(',')]))

In [3]:
input_midi = "../data/test_melody_hatikvah(israel)_4_1_minor_1.mid"
#melody, tempo_interval, meter, key, tonality, first_on_beat = midi_to_array(input_midi)
melody, tempo_interval = midi_to_array_quick(input_midi)
filename = str.split(str.split(input_midi,'/')[-1],'_')
first_on_beat = int(str.split(filename[-1],'.')[0])
tonality = filename[-2]
key = int(filename[-3])
meter = int(filename[-4])
song_title = '_'.join(filename[2:-4])
musical_corpus = []
musical_corpus.append(MusicalWorkInput(song_title, meter, key, tonality, first_on_beat, list(filter(None,melody[0]))))

In [4]:
# Importing Chord Vocabulary
chord_df_major = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
chord_df_minor = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab_major, chord_vocab_minor = [], []
for index, name, note_intervals in chord_df_major.itertuples():
    chord_vocab_major.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))
for index, name, note_intervals in chord_df_minor.itertuples():
    chord_vocab_minor.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))

In [5]:
# Defining penalties for chord progression
penalties_chord_progression_major = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
penalties_chord_progression_minor = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)
penalties_chord_progression_major = dict(penalties_chord_progression_major.stack())
penalties_chord_progression_minor = dict(penalties_chord_progression_minor.stack())

In [6]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights.csv")

# Defining dictionary of weights for each soft constraint option:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
assert sum(v for v in soft_constraint_w_weights.values() if v > 0) == 100

In [7]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing', 'incomplete chord', 'chord distribution']
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'incomplete chord', 'voice crossing', 'voice range',
                           'second inversion', 'first inversion', 'chord distribution']

In [8]:
# Defining which hard constraints to use
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords',
                                     'voice crossing', 'parallel movement',
                                     'chord spacing', 'incomplete chord'] else False for x in hard_constraint_options}

In [9]:
# Model
for musical_work in [musical_corpus[-1]]:

    print('*'*20)
    print('Work: {}'.format(musical_work.title))
    print('*'*20)
    
    if musical_work.tonality == 'major':
        penalties_chord_progression = penalties_chord_progression_major
        chord_vocab = chord_vocab_major
    else:
        penalties_chord_progression = penalties_chord_progression_minor
        chord_vocab = chord_vocab_minor        
    
    #Solving Model
    ga_model = src.ga_model.GAmodel(musical_work, chord_vocab, max_generation=200, population_size=100,
                                    hard_constraints=hard_constraints,
                                    soft_constraint_w_weights=soft_constraint_w_weights,
                                    chord_progression_penalties=penalties_chord_progression,
                                    no_of_mutations=musical_work.melody_len//8, mutation_probability=[0.1, 0.9])
    solution, midi_array, progress_array = ga_model.solve()

********************
Work: hatikvah(israel)
********************
0 (0.62, 29605.0)
1 (0.66, 29605.0)
2 (0.68, 29605.0)
3 (0.72, 29605.0)
4 (0.65, 29605.0)
5 (0.67, 29605.0)
6 (0.66, 29605.0)
7 (0.7, 29605.0)
8 (0.81, 29605.0)
9 (0.65, 29605.0)
10 (0.71, 29605.0)
11 (0.68, 29605.0)
12 (0.7, 29605.0)
13 (0.67, 29605.0)
14 (0.63, 29605.0)
15 (0.65, 29605.0)
16 (0.71, 29605.0)
17 (0.69, 29605.0)
18 (0.68, 29605.0)
19 (0.71, 29605.0)
20 (0.67, 29605.0)
21 (0.64, 29605.0)
22 (0.68, 29605.0)
23 (0.66, 29605.0)
24 (0.7, 29605.0)
25 (0.83, 29605.0)
26 (0.82, 29605.0)
27 (0.64, 29605.0)
28 (0.65, 29605.0)
29 (0.72, 29605.0)
30 (0.66, 29605.0)
31 (0.73, 29605.0)
32 (0.72, 29605.0)
33 (0.62, 29605.0)
34 (0.62, 29605.0)
35 (0.7, 29605.0)
36 (0.64, 29605.0)
37 (0.77, 29605.0)
38 (0.69, 29605.0)
39 (0.62, 29605.0)
40 (0.66, 29605.0)
41 (0.65, 29605.0)
42 (0.67, 29605.0)
43 (0.74, 29605.0)
44 (0.68, 29605.0)
45 (0.7, 29605.0)
46 (0.74, 29605.0)
47 (0.67, 29605.0)
48 (0.69, 29605.0)
49 (0.67, 29605.0)
